# Projeto: Séries temporais - MBA

Observações:

1. Para cada questão, crie novas células para efetuar seus códigos.

2. Nomear variáveis de acordo com o contexto da questão.

3. Caso necessário, utilize novas caixas de textos para deixar comentários.

Prof. Jorge Luiz

jorgearaujo@unifor.br

1. Carregue o arquivo "DENGUE_MBA_ST.csv". Faça o que se pede:

a. Quantos casos de dengue há em cada ano para a cidade de Fortaleza?

b. Crie gráficos temporal apresentando algumas médias móveis para o número de casos de Dengue em Fortaleza.

c. O dado é estacionário?

d. O dado apresenta uma distribuição normal?

e. Escolha um periodo para treinar o algum modelo de ML e faça a predição de algumas semanas posteriores. Crie um gráfico demonstrando o predição. Qual é a melhor forma de dividir o dado?

f. Para 2011, as séries temporais de casos de dengue entre bairros são semelhantes? Calcule a correlação entre os pares de bairros e verifique o valor médio.




# Dica de Solução

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.stattools import kpss
from scipy.stats import shapiro
from statsmodels.tsa.arima.model import ARIMA

## Questão 1

In [ ]:
df=pd.read_csv('DENGUE_MBA_ST.csv')

In [ ]:
df

,ANO,SEMANA_DO_ANO,AEROLANDIA,AEROPORTO,ALDEOTA,ALTO DA BALANCA,ALVARO WEYNE,AMADEU FURTADO,ANCURI,ANTONIO BEZERRA,...,VICENTE PINZON,VILA ELLERY,VILA PERY,VILA UNIAO,VILA VELHA,ALAGADICO,GENTILANDIA,MATA GALINHA,PAN-AMERICANO,DATA
0,2011,1,0,0,1,0,0,0,3,0,...,1,0,1,0,2,0.0,0.0,0.0,0.0,2011-01-09
1,2011,2,0,0,0,0,1,0,1,0,...,1,1,5,1,4,0.0,0.0,0.0,0.0,2011-01-16
2,2011,3,0,0,2,1,1,0,7,1,...,6,0,6,4,2,0.0,0.0,0.0,0.0,2011-01-23
3,2011,4,0,0,0,0,2,0,2,0,...,6,2,8,1,0,0.0,0.0,0.0,0.0,2011-01-30
4,2011,5,0,0,0,1,4,0,2,0,...,6,2,2,5,1,0.0,0.0,0.0,0.0,2011-02-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,2020,49,0,0,0,0,1,0,2,0,...,0,0,1,0,0,0.0,0.0,0.0,0.0,2020-12-06
519,2020,50,0,0,0,0,1,0,1,0,...,0,0,0,0,2,0.0,0.0,0.0,0.0,2020-12-13
520,2020,51,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0.0,0.0,0.0,0.0,2020-12-20
521,2020,52,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0.0,0.0,0.0,0.0,2020-12-27


In [ ]:
#transformando datetime
df['DATA']=pd.to_datetime(df['DATA'])

In [ ]:
df.columns

Index(['ANO', 'SEMANA_DO_ANO', 'AEROLANDIA', 'AEROPORTO', 'ALDEOTA',
       'ALTO DA BALANCA', 'ALVARO WEYNE', 'AMADEU FURTADO', 'ANCURI',
       'ANTONIO BEZERRA',
       ...
       'VICENTE PINZON', 'VILA ELLERY', 'VILA PERY', 'VILA UNIAO',
       'VILA VELHA', 'ALAGADICO', 'GENTILANDIA', 'MATA GALINHA',
       'PAN-AMERICANO', 'DATA'],
      dtype='object', length=122)

In [ ]:
# Escolhenado apenas os bairros
df.drop(['ANO','SEMANA_DO_ANO','DATA'],axis=1)

,AEROLANDIA,AEROPORTO,ALDEOTA,ALTO DA BALANCA,ALVARO WEYNE,AMADEU FURTADO,ANCURI,ANTONIO BEZERRA,ARRAIAL MOURA BRASIL,AUTRAN NUNES,...,VARJOTA,VICENTE PINZON,VILA ELLERY,VILA PERY,VILA UNIAO,VILA VELHA,ALAGADICO,GENTILANDIA,MATA GALINHA,PAN-AMERICANO
0,0,0,1,0,0,0,3,0,0,0,...,0,1,0,1,0,2,0.0,0.0,0.0,0.0
1,0,0,0,0,1,0,1,0,1,0,...,0,1,1,5,1,4,0.0,0.0,0.0,0.0
2,0,0,2,1,1,0,7,1,4,0,...,1,6,0,6,4,2,0.0,0.0,0.0,0.0
3,0,0,0,0,2,0,2,0,1,1,...,0,6,2,8,1,0,0.0,0.0,0.0,0.0
4,0,0,0,1,4,0,2,0,2,0,...,1,6,2,2,5,1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,0,0,0,0,1,0,2,0,0,0,...,0,0,0,1,0,0,0.0,0.0,0.0,0.0
519,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,2,0.0,0.0,0.0,0.0
520,0,0,0,0,0,0,0,0,1,0,...,0,0,0,2,0,0,0.0,0.0,0.0,0.0
521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0.0,0.0,0.0,0.0


In [ ]:
# capturando todos os nomes dos bairros
bairroname=df.drop(['ANO','SEMANA_DO_ANO','DATA'],axis=1).columns

In [ ]:
# Dado de Fortaleza
Fortaleza=pd.Series(df[bairroname].sum(axis=1).values,index=df['DATA'].values)

In [ ]:
Fortaleza

2011-01-09    200.0
2011-01-16    296.0
2011-01-23    288.0
2011-01-30    297.0
2011-02-06    341.0
              ...  
2020-12-06     29.0
2020-12-13     26.0
2020-12-20     36.0
2020-12-27     21.0
2020-12-31     17.0
Length: 523, dtype: float64